In [314]:
import numpy as np
import heapq

In [315]:
def computeMinPath(graph, numVertices):
  minPath = np.zeros((numVertices, 2))
  for i in range(numVertices):
    edgeWeights = graph[i]
    
    minEdge = np.min(edgeWeights)
    secondMinEdge = np.partition(edgeWeights, 1)[1]

    minPath[i] = [minEdge, secondMinEdge]

  return minPath
      
def getOnePath(graph, numVertices):
  path = np.array([0])
  cost = 0
  for i in range(1, numVertices):
    cost += graph[path[-1]][i]
    path = np.concatenate([path, [i]]).astype(int)
  cost += graph[path[-1]][0]
  path = np.concatenate([path, [0]]).astype(int)
  return path, cost

class Node:
  def __init__(self, graph, minPath, prevNode=None, nextVertice=None):
    if prevNode is None or nextVertice is None:
      self.level = 1
      self.path = [0]
      self.cost = 0
      self.bound = bound(graph, minPath, 0)
      return

    self.level = prevNode.level + 1
    self.path = np.concatenate([prevNode.path, [nextVertice]]).astype(int)
    if len(prevNode.path) == 0:
      self.cost = 0
    else:
      self.cost = prevNode.cost + graph[prevNode.path[-1]][nextVertice]
    self.bound = bound(graph, minPath, nextVertice, prevNode)

  def __lt__(self, other):
    return self.bound < other.bound
  
  def __gt__(self, other):
    return self.bound > other.bound
  
  def __eq__(self, other):
    return self.bound == other.bound
  
  def __str__(self) -> str:
    return f"Path: {self.path} Cost: {self.cost} Bound: {self.bound}"


def bound(graph, minPath, nextVertice, prevNode=None):
  if prevNode is None:
    return np.sum(minPath) / 2
  
  cost = graph[prevNode.path[-1]][nextVertice]

  bound = prevNode.bound * 2

  if cost >= minPath[nextVertice][1]:
    bound -= minPath[nextVertice][1]
    bound += cost
  
  if cost >= minPath[prevNode.path[-1]][1]:
    bound -= minPath[prevNode.path[-1]][1]
    bound += cost

  return bound / 2


In [316]:
def branchAndBoundTSP(graph, numVertices):
  minPath = computeMinPath(graph, numVertices)

  root = Node(graph, minPath, None, 0)
  queue = [root]
  heapq.heapify(queue)

  solution, best = getOnePath(graph, numVertices)

  while queue:
    node = heapq.heappop(queue)
    
    if node.level >= numVertices:
      finalNode = Node(graph, minPath, node, 0)
      if finalNode.cost < best:
        best = finalNode.cost
        solution = finalNode.path
    
    elif node.bound < best:
      unvisited = np.setdiff1d(np.arange(numVertices), node.path)
      for k in unvisited:
        newNode = Node(graph, minPath, node, k)
        if newNode.bound < best and graph[node.path[-1]][k] != np.inf:
          heapq.heappush(queue, newNode)

  return solution, best

In [317]:
def generateMST(graph, numVertices):
  included = [False] * numVertices
  included[0] = True

  cost = 0
  edges = []
  numVisited = 1

  while numVisited < numVertices:
    minEdge = (float('inf'), None, None) 

    for i in range(numVertices):
      if included[i]:
        for j in range(numVertices):
          if not included[j] and graph[i][j] < minEdge[0]:
            minEdge = (graph[i][j], i, j)

    cost, startVertex, endVertex = minEdge
    included[endVertex] = True
    cost += cost
    edges.append((startVertex, endVertex))
    numVisited += 1

  mstAdjMatrix = np.full((numVertices, numVertices), np.inf)
  for edge in edges:
    mstAdjMatrix[edge[0]][edge[1]] = mstAdjMatrix[edge[1]][edge[0]] = graph[edge[0]][edge[1]]

  return mstAdjMatrix, cost

In [318]:
def preorderWalkAux(adjMatrix, startVertex, visited, order):
    order.append(startVertex)
    visited[startVertex] = True

    for neighbor in range(len(adjMatrix)):
      if adjMatrix[startVertex][neighbor] != np.inf and not visited[neighbor]:
        preorderWalkAux(adjMatrix, neighbor, visited, order)

def preorderWalk(adjMatrix, start_vertex=0):
    numVertices = len(adjMatrix)
    visited = [False] * numVertices
    order = []

    preorderWalkAux(adjMatrix, start_vertex, visited, order)

    return order

In [319]:
def approximateTwiceAroundTheTree(graph, numVertices):
  mstAdjMatrix, mstCost = generateMST(graph, numVertices)

  preorder = preorderWalk(mstAdjMatrix)
  tour = preorder + [preorder[0]]

  tourCost = sum(graph[tour[i]][tour[i + 1]] for i in range(len(tour) - 1))

  return tour, tourCost

In [320]:
def findMinimumWeightMatching(mstAdjMatrix, oddVertices):
  matchingEdges = set()
  matchedRows = set()
  matchedCols = set()

  for vertex in oddVertices:
    minWeight = float('inf')
    minCol = None

    for col in oddVertices:
      if vertex != col and col not in matchedCols and mstAdjMatrix[vertex][col] < minWeight:
        minWeight = mstAdjMatrix[vertex][col]
        minCol = col

    matchingEdges.add((vertex, minCol))
    matchedRows.add(vertex)
    matchedCols.add(minCol)

  return matchingEdges

In [321]:
def eulerianCircuit(graph):
  circuit = [0]
  while True:
    neighbors = np.nonzero(graph[circuit[-1]])
    unvisited_neighbors = [neighbor for neighbor in neighbors[0] if neighbor not in circuit]
    if not unvisited_neighbors:
      break
    circuit.append(unvisited_neighbors[0])
  return circuit

In [322]:
def christofides(graph, numVertices):
  mstAdjMatrix, mstCost = generateMST(graph, numVertices)

  oddVertices = [vertex for vertex, degree in enumerate(np.sum(mstAdjMatrix != np.inf, axis=1)) if degree % 2 != 0]

  matchingEdges = findMinimumWeightMatching(mstAdjMatrix, oddVertices)

  multigraph = mstAdjMatrix.copy()

  for edge in matchingEdges:
    if edge[0] < numVertices and edge[1] < numVertices:
      multigraph[edge[0]][edge[1]] = multigraph[edge[1]][edge[0]] = graph[edge[0]][edge[1]]


  eulerianCircuit = eulerianCircuit(multigraph)

  hamiltonianCircuit = list(dict.fromkeys(eulerianCircuit))

  # Add the last vertex to complete the circuit
  hamiltonianCircuit.append(hamiltonianCircuit[0])

  # Calculate the cost of the Hamiltonian circuit
  tourCost = sum(graph[hamiltonianCircuit[i]][hamiltonianCircuit[i + 1]] for i in range(len(hamiltonianCircuit) - 1))

  return hamiltonianCircuit, tourCost

In [323]:
numVertices = 4
graph = np.full((numVertices, numVertices), np.inf)
graph = np.array([
       [0, 10, 15, 20],
       [10, 0, 35, 25],
       [15, 35, 0, 30],
       [20, 25, 30, 0]
       ])
solution, best = branchAndBoundTSP(graph, numVertices)

print(f"Solution: {solution} Cost: {best}")

Solution: [0 1 3 2 0] Cost: 80


In [324]:
print(generateMST(graph, numVertices))

print(preorderWalk(graph, 0))

print("Twice around the tree", approximateTwiceAroundTheTree(graph, numVertices))

(array([[inf, 10., 15., 20.],
       [10., inf, inf, inf],
       [15., inf, inf, inf],
       [20., inf, inf, inf]]), 40)
[0, 1, 2, 3]
Twice around the tree ([0, 1, 2, 3, 0], 95)


In [325]:
def preorder_walk(graph, start_vertex, visited):
    print(start_vertex, end=" ")
    visited[start_vertex] = True

    for neighbor in range(len(graph)):
      if graph[start_vertex][neighbor] != np.inf and not visited[neighbor]:
        preorder_walk(graph, neighbor, visited)

In [326]:
class Point:
  def __init__(self, id, x, y):
    self.id = id
    self.x = x
    self.y = y

  def distance(self, point):
    return np.sqrt((self.x - point.x)**2 + (self.y - point.y)**2)
  
  def __str__(self) -> str:
    return f'Point {self.id} ({self.x}, {self.y})'

In [327]:
# function to read a file and return a list of lists
def readFile(file_name):
    with open(file_name) as f:
        content = f.readlines()
    content = [x.strip() for x in content if x[0].isdigit()]
    return content


In [328]:
# each element of the content list is a string with the id, x and y coordinates
# we need to split each line and create a Point object

content = readFile('berlin52.tsp')

points = []
for line in content:
  id, x, y = line.split(' ')
  points.append(Point(int(id)-1, float(x), float(y)))

for p in points:
  print(p)

Point 0 (565.0, 575.0)
Point 1 (25.0, 185.0)
Point 2 (345.0, 750.0)
Point 3 (945.0, 685.0)
Point 4 (845.0, 655.0)
Point 5 (880.0, 660.0)
Point 6 (25.0, 230.0)
Point 7 (525.0, 1000.0)
Point 8 (580.0, 1175.0)
Point 9 (650.0, 1130.0)
Point 10 (1605.0, 620.0)
Point 11 (1220.0, 580.0)


In [329]:
numVertices = len(points)
graph = np.full((numVertices, numVertices), np.inf)

for p in points:
  for q in points:
    if p.id != q.id:
      graph[p.id][q.id] = p.distance(q)


In [330]:
print('Branch and bound', branchAndBoundTSP(graph, numVertices))

Branch and bound (array([ 0,  1,  6,  2,  7,  8,  9, 10, 11,  3,  5,  4,  0]), 4056.6809443061275)


In [331]:
print("Twice around the tree", approximateTwiceAroundTheTree(graph, numVertices))

Twice around the tree ([0, 2, 6, 1, 7, 9, 8, 4, 5, 3, 11, 10, 0], 4567.439293000949)


In [332]:
print("Christofides", christofides(graph, numVertices))

TypeError: '<' not supported between instances of 'NoneType' and 'int'